In [1]:
# Ingnore warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Import libraries
import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import math

import tkinter as tk
from tkinter import messagebox
from plyer import notification


# Leer archivo de excel
s_filepath=r'C:\AnalisisElectrico\Scripts_Py_Dig\Salida_Dig.xlsx'
df_data=pd.read_excel(s_filepath, sheet_name='Data')

s_Fecha=df_data.loc[0,'Valor']

iPini=df_data.loc[3,'Valor']
iPfin=df_data.loc[4,'Valor']

ano=s_Fecha.year
mes=s_Fecha.month
dia=s_Fecha.day


In [2]:
# Function to show a notification
def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        app_icon=None,  # e.g., 'C:\\path\\to\\icon_32x32.ico'
        timeout=30,  # seconds
    )

In [3]:
# Determinar ruta para obtener la demanda de la fecha
s_filepath=Path(r'C:\Información XM\DEMANDAS\Pronostico Oficial\\')
file=r'PRON_BARRA' + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
filepath=s_filepath.joinpath(s_filepath,str(ano) + '-' + "{:02d}".format(mes),file)


In [5]:
nameCol=['UCP','BARRA','kV','Variable']
perCol=[]

for i in range(1,25):
    perCol.append('P'+str(i))
    nameCol.append('P'+str(i))


In [6]:
# Lectura del archivo
df_data = pd.read_csv(filepath, sep=',', header=None, names=nameCol, encoding="ISO-8859-1")

In [7]:
df_data.head(5)

,UCP,BARRA,kV,Variable,P1,P2,P3,P4,P5,P6,...,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24
0,Uandaki,DONCELL1,115,Ener.Act.,11.044425,10.456768,10.201758,10.093112,10.597932,10.946307,...,16.614587,16.487653,16.973610,16.819606,18.024244,18.535839,17.289041,15.493855,13.729114,12.545216
1,Uandaki,DONCELL1,115,Ener.Reac.,1.578531,1.328158,1.280878,1.210042,1.270564,1.400891,...,5.553551,5.465894,5.501477,5.244355,5.240287,5.039605,4.430406,3.553763,2.590526,2.193947
2,Uandaki,DONCELL1,115,Perd.PSTR,0.168189,0.159240,0.155357,0.153702,0.161390,0.166695,...,0.253014,0.251081,0.258481,0.256136,0.274481,0.282272,0.263285,0.235947,0.209073,0.191044
3,Uandaki,FLORENC1,115,Ener.Act.,20.467858,19.031312,18.634689,17.922298,17.378976,19.227096,...,33.357102,34.348388,33.448544,31.971059,31.882291,32.719107,31.673155,29.308084,25.882861,23.109796
4,Uandaki,FLORENC1,115,Ener.Reac.,9.483366,8.921963,8.619462,8.490424,7.642887,9.255944,...,16.659637,16.642493,16.786602,15.175064,15.050837,15.610438,14.214981,13.461357,12.415902,10.977649


In [9]:
# Get main path and other folders
s_mainpath=Path.cwd()
s_parentpath=s_mainpath.parent
s_folderpath='AnalisisMensual'
s_file='Mapeo_SIO_Dig.xlsx'
s_filepath=s_parentpath.joinpath(s_folderpath,s_file)

df_MapLoad=pd.read_excel(s_filepath, sheet_name='Map_Load')

In [10]:
df_MapLoad['Name']=df_MapLoad['Name'].str.replace('lod ','')
df_MapLoad['Name']=df_MapLoad['Name'].str.replace('Lod ','')
l_colums=list(df_data.columns)
l_colums.append('Fkey')
df_dat_Fkey=pd.merge(df_data,df_MapLoad,left_on=['BARRA'],right_on=['Name'],how='left')[l_colums]

In [31]:
df_data_Pro=pd.DataFrame(df_data['BARRA'])
df_data_Pro=df_data_Pro.drop_duplicates(subset=['BARRA'])

In [36]:
df_Dig_NoPron=pd.merge(df_MapLoad,df_data_Pro,left_on=['Name'],right_on=['BARRA'],how='left')
df_Dig_NoPron.fillna('No tiene Pron', inplace=True)

In [37]:
df_Dig_NoPron.to_csv('DataBarra.csv')

In [22]:
df_filtrado = df_dat_Fkey[df_dat_Fkey['Fkey'].isna()]

nrow=df_filtrado.shape[0]

if nrow>0:

    sum_per=pd.DataFrame(df_filtrado[perCol].sum(),columns=['Value'])
    sum_per_g0=sum_per[(sum_per.Value>0)].shape[0]

    if sum_per_g0>0:
        s_msj='Hay Barras con pronóstico mayor que cero y sin Fkey, se debe revisar porque puede causar inconsitencia'
        print (s_msj)
        print (df_filtrado)
        show_notification("Mensaje de Advertencia", s_msj)

df_dat_Fkey=df_dat_Fkey.drop(df_filtrado.index)
df_dat_Fkey.reset_index(drop=True,inplace=True)

In [23]:
def EncabezadoDGS(FileName):
    file = open(FileName,'w')
    file.write('$$General;ID(a:40);Descr(a:40);Val(a:40)\n'\
                '1;Version;5.0\n')
    file.close()

def ElementosElmLoad(FileName):
    file = open(FileName,'a+')
    file.write('$$ElmLod;ID(a:40);outserv(i);plini(r);qlini(r)\n')
    file.close()

In [24]:
def WriteElm(FileName, l_loads, df_Act, df_React,t):

    l_dgs=[]
    for lod in l_loads:
        
        df_Aux=df_Act.loc[(df_Act.BARRA==lod)]
        nrow=df_Aux.shape[0]

        if nrow>1:
            s_msj='El elemento ' + str(lod) + ' tiene más de un registro en demanda, se debe revisar'
            print(s_msj)
            show_notification("Mensaje de Advertencia", s_msj)
        else:
            sFkey=df_Act.loc[(df_Act.BARRA==lod),'Fkey'].values[0]
            sAct=str(df_Act.loc[(df_Act.BARRA==lod),'P' + str(t)].values[0])
            sReact=str(df_React.loc[(df_React.BARRA==lod),'P' + str(t)].values[0])
            sStr='##' + sFkey + ';0;' + sAct + ';' + sReact
            l_dgs.append(sStr)

    outPutFile=pd.DataFrame(l_dgs)
    outPutFile.to_csv(FileName, index=False, header=None, mode='a+') 

In [25]:
l_periodos=[i for i in range(iPini,iPfin+1)]

RutaDGS=r"C:\AnalisisElectrico\DGS_Demanda"

l_loads=list(df_dat_Fkey['BARRA'].unique())

df_Act=df_dat_Fkey[(df_dat_Fkey.Variable=='Ener.Act.')]
df_React=df_dat_Fkey[(df_dat_Fkey.Variable=='Ener.Reac.')]

for t in l_periodos:

    FileName = RutaDGS + "\P{:02}".format(int(t)) + '.dgs'
    EncabezadoDGS(FileName)

    ElementosElmLoad(FileName)
    WriteElm(FileName, l_loads, df_Act[['BARRA','P' + str(t),'Fkey']], df_React[['BARRA','P' + str(t)]],t)

    